# 🔗 Law-Linking Pipeline (Colab Edition)

LDVS→BM25→(Ko‑Legal‑SBERT) Re‑rank→NLI Entailment→Fact Box→Confidence

260개 법령 JSON 카탈로그(`Law_Extract_260.json`) 기반 법률 후보 자동 채택

**구성:**
1) 의존성 설치 → 2) 설정값 정의 → 3) 데이터 클래스/유틸 → 4) BM25 → 5) SBERT 재랭킹 → 6) NLI → 7) 법제처 API 팩트박스 → 8) 오케스트레이션 → 9) 예시 실행

In [1]:
import sys
!pip install rank_bm25 sentence-transformers transformers torch lxml requests

In [2]:
# 1) Imports & Config
import os, re, json, math
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import requests
from lxml import etree
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util

class PipelineConfig:
    """파이프라인 설정값: 모델명/탑K/임계값/API키 등"""
    def __init__(self,
                 law_json_path: str = "Law_Extract_260.json", # "../data/Law_Extract_260.json"
                 legal_sbert_model: str = "snunlp/KR-SBERT-V40K-klueNLI-augSTS",
                 nli_model: str = "joeddav/xlm-roberta-large-xnli",
                 device: Optional[str] = None,
                 bm25_topk: int = 50,
                 rerank_topk: int = 10,
                 auto_accept: float = 0.75,
                 human_review: Tuple[float, float] = (0.55, 0.75),
                 use_law_api: bool = True,
                 law_api_oc: Optional[str] = None):
        self.law_json_path = law_json_path
        self.legal_sbert_model = legal_sbert_model
        self.nli_model = nli_model
        self.device = device
        self.bm25_topk = bm25_topk
        self.rerank_topk = rerank_topk
        self.auto_accept = auto_accept
        self.human_review = human_review
        self.use_law_api = use_law_api
        self.law_api_oc = law_api_oc or os.getenv('LAW_OC')

cfg = PipelineConfig()

In [3]:
# 2) Data classes & Utilities
# - LawEntry: 카탈로그의 한 법령 레코드
# - Cluster  : LDVS 통과 군집 입력 (키워드/대표문)
# - Candidate: 검색/재랭킹/NLI/팩트박스/신뢰도 정보를 모은 후보
import json
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
import re
from lxml import etree

@dataclass
class LawEntry:
    law_id: Optional[str]
    official_name: str
    law_type: Optional[str]
    promulgation_no: Optional[str]
    promulgation_date: Optional[str]
    effective_date: Optional[str]
    raw_json: Optional[Dict[str, Any]]

@dataclass
class Cluster:
    cluster_id: int
    ldvs_pass: bool
    keywords: List[str]
    rep_texts: List[str]
    summary_text: Optional[str] = None

@dataclass
class Candidate:
    law: LawEntry
    bm25_score: float
    sbert_score: Optional[float] = None
    nli_entail_prob: Optional[float] = None
    factbox: Optional[Dict[str, Any]] = None
    conf: Optional[float] = None

def normalize_text(s: str) -> str:
    """공백 정규화"""
    import re
    return re.sub(r"\s+", " ", s or "").strip()

# --- 진단용: JSON 최상위 구조를 간단히 보여줌
def _peek_json_shape(obj: Any, prefix: str = "", max_depth: int = 2):
    if max_depth < 0:
        print(prefix + "…")
        return
    if isinstance(obj, dict):
        print(prefix + f"dict(keys={list(obj.keys())[:8]})")
        for k, v in list(obj.items())[:5]:
            print(prefix + f"  - {k}: {type(v).__name__}")
        # 한 단계만 더 파고들어봐
        for k, v in list(obj.items())[:1]:
            _peek_json_shape(v, prefix + "    ", max_depth-1)
    elif isinstance(obj, list):
        print(prefix + f"list(len={len(obj)})")
        if obj:
            print(prefix + f"  [0]: {type(obj[0]).__name__}")
            _peek_json_shape(obj[0], prefix + "    ", max_depth-1)
    else:
        print(prefix + type(obj).__name__)

def _find_table_rows(obj: Any, max_scan_nodes: int = 10000) -> Optional[List[List[Any]]]:
    """
    dict 트리를 DFS로 훑으면서 'list[list]' 테이블 후보를 찾는다.
    - 행 길이>=2, 최소 3~5행 이상 등 간단한 휴리스틱 적용
    """
    stack = [obj]
    scanned = 0
    while stack:
        cur = stack.pop()
        scanned += 1
        if scanned > max_scan_nodes:
            break
        if isinstance(cur, dict):
            for v in cur.values():
                stack.append(v)
        elif isinstance(cur, list):
            if cur and isinstance(cur[0], list):
                # 테이블로 볼만한지 판단(행 수/열 수 대략 체크)
                row0 = cur[0]
                if len(row0) >= 2 and len(cur) >= 1:
                    return cur
            # 아니면 더 파고들기
            for v in cur[:10]:
                stack.append(v)
    return None

def _guess_raw_json_idx(row: List[Any]) -> Optional[int]:
    """
    한 행 안에서 '원시 JSON'처럼 보이는 컬럼의 인덱스를 유추한다.
    - 문자열이면서 '{'로 시작하거나 'orgdocXmlCtt' 같은 키워드가 들어 있으면 가중치↑
    - 여러 후보가 있으면 가장 가능성 높은 것 선택
    """
    best_idx, best_score = None, -1
    for i, val in enumerate(row):
        score = 0
        if isinstance(val, str):
            s = val.strip()
            if s.startswith("{") or s.startswith("["):
                score += 2
            if "orgdocXmlCtt" in s or "법령ID" in s or "법령명" in s or "lawHanNm" in s:
                score += 2
            if len(s) > 200:  # 꽤 긴 텍스트면 가중
                score += 1
        elif isinstance(val, dict):
            score += 3  # 이미 파싱된 dict면 강력 후보
        if score > best_score:
            best_score, best_idx = score, i
    return best_idx if best_score >= 0 else None

def load_law_catalog(path: str) -> List[LawEntry]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 0) 구조 진단(한 번만 프린트)
    print("🔍 JSON top-level shape:")
    _peek_json_shape(data)

    # 1) 테이블(rows: list[list]) 탐색
    rows = None
    if isinstance(data, dict) and isinstance(data.get("rows"), list) and data["rows"] and isinstance(data["rows"][0], list):
        rows = data["rows"]
        found_where = "top-level 'rows'"
    else:
        rows = _find_table_rows(data)
        found_where = "nested search"

    if not rows:
        print("⚠️ list[list] 테이블을 찾지 못했습니다. 파일 구조를 확인하세요.")
        return []

    print(f"📎 Found table ({len(rows)} rows) via {found_where}")

    entries: List[LawEntry] = []
    for row in rows:
        # 유연한 컬럼 맵핑(가능하면 ‘두 번째=법령명’ 가정 유지)
        rid   = str(row[0]) if len(row) > 0 else ""
        name  = str(row[1]) if len(row) > 1 else ""   # ★ 너가 원하는 “두번째 정보(법령명)” 사용
        ltype = str(row[2]) if len(row) > 2 and row[2] not in (None, "") else None
        prno  = str(row[3]) if len(row) > 3 and row[3] not in (None, "") else None
        prymd = str(row[4]) if len(row) > 4 and row[4] not in (None, "") else None
        efymd = str(row[5]) if len(row) > 5 and row[5] not in (None, "") else None

        # raw_json 컬럼 자동 유추
        raw_idx = _guess_raw_json_idx(row)
        rj = row[raw_idx] if (raw_idx is not None and raw_idx < len(row)) else None

        if isinstance(rj, str):
            try:
                rj = json.loads(rj)
            except Exception:
                rj = {"raw": rj}

        if not name:
            continue

        entries.append(LawEntry(
            law_id=rid,
            official_name=name,
            law_type=ltype,
            promulgation_no=prno,
            promulgation_date=prymd,
            effective_date=efymd,
            raw_json=rj if isinstance(rj, dict) else {"raw_json": rj}
        ))

    print(f"📚 Loaded laws: {len(entries)}")
    return entries

# 스모크 테스트
_preview = load_law_catalog(cfg.law_json_path)
print("🧪 First item:", _preview[0].official_name if _preview else "(none)")

def build_law_catalog_text(law: LawEntry) -> str:
    parts = [law.official_name or ""]
    for x in [law.law_type, law.promulgation_no, law.promulgation_date, law.effective_date]:
        if x: parts.append(str(x))
    rj = law.raw_json or {}
    for k in ["lawKndCdnm", "attblCtt", "splprvCtt", "orgdocXmlCtt", "htagCtt"]:
        v = rj.get(k)
        if isinstance(v, str) and v.strip():
            parts.append(v)
    xml_s = rj.get("orgdocXmlCtt")
    if isinstance(xml_s, str) and xml_s.strip().startswith("<?xml"):
        try:
            root = etree.fromstring(xml_s.encode("utf-8"))
            text = " ".join(root.itertext())
            parts.append(text[:2000])
        except Exception:
            pass
    return normalize_text(" ".join(parts))

def sentences_from_keywords(keywords: List[str], k: int = 20) -> str:
    """Top‑k 키워드를 간단한 문장 형태로 변환."""
    toks = [kw for kw in keywords[:k] if kw]
    if not toks:
        return ""
    return f"핵심 키워드: {', '.join(toks)}."

def filter_ldvs_clusters(clusters: List[Cluster]) -> List[Cluster]:
    return [c for c in clusters if c.ldvs_pass]

🔍 JSON top-level shape:
list(len=1)
  [0]: dict
    dict(keys=['stmt', 'header', 'rows'])
      - stmt: str
      - header: list
      - rows: list
        str
📎 Found table (200 rows) via nested search
📚 Loaded laws: 200
🧪 First item: 법제처 직제 시행규칙


In [4]:
# 3) Retrieval & Reasoning Components
# 3.1 BM25 카탈로그
class BM25LawCatalog:
    def __init__(self, laws: List[LawEntry]):
        self.laws = laws
        self.docs = [build_law_catalog_text(l) for l in laws]
        self.tokenized = [self._tokenize(d) for d in self.docs]
        self.bm25 = BM25Okapi(self.tokenized)

    @staticmethod
    def _tokenize(s: str) -> List[str]:
        # 간단한 토크나이저(한/영/숫자) — 필요하면 MeCab/Okt로 대체
        s = s.lower()
        s = re.sub(r"[^0-9a-z가-힣]+", " ", s)
        return s.split()

    def search(self, query: str, topk: int = 50) -> List[Tuple[int, float]]:
        q_toks = self._tokenize(query)
        scores = self.bm25.get_scores(q_toks)
        idx = np.argsort(scores)[::-1][:topk]
        return [(int(i), float(scores[i])) for i in idx]

# 3.2 SBERT 재랭커
class ReRanker:
    def __init__(self, model_name: str, device: Optional[str] = None):
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = SentenceTransformer(model_name, device=device)

    def rerank(self, query: str, candidates: List[Tuple[int, float]], corpus_texts: List[str], topk: int = 10):
        cand_ix = [ix for ix, _ in candidates]
        cand_texts = [corpus_texts[i] for i in cand_ix]
        q_emb = self.model.encode([query], convert_to_tensor=True, normalize_embeddings=True)
        c_emb = self.model.encode(cand_texts, convert_to_tensor=True, batch_size=64, normalize_embeddings=True)
        sims = util.cos_sim(q_emb, c_emb).cpu().numpy().ravel().tolist()
        ranks = sorted(list(zip(cand_ix, sims)), key=lambda x: x[1], reverse=True)[:topk]
        return ranks  # (idx, cosine)

# 3.3 NLI 엔테일 확률 추정기
class NLIEstimator:
    def __init__(self, model_name: str = "joeddav/xlm-roberta-large-xnli", device: Optional[str]=None):
        from transformers import AutoModelForSequenceClassification, AutoTokenizer
        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
        self.device = device
        # id2label에서 entail/contradiction/neutral 인덱스 파악(모델마다 다름)
        id2label = self.model.config.id2label
        self.idx_entail = int([k for k, v in id2label.items() if v.lower().startswith('entail')][0])
        self.idx_contra = int([k for k, v in id2label.items() if v.lower().startswith('contrad')][0])
        self.idx_neutral = int([k for k, v in id2label.items() if v.lower().startswith('neutral')][0])

    @torch.no_grad()
    def entail_prob(self, premise: str, hypothesis: str) -> float:
        inp = self.tokenizer(premise, hypothesis, truncation=True, padding=True, max_length=512, return_tensors='pt').to(self.device)
        logits = self.model(**inp).logits
        probs = torch.softmax(logits, dim=-1)[0].detach().cpu().numpy()
        return float(probs[self.idx_entail])

# 3.4 법제처 DRF 간이 래퍼(팩트박스)
class LawGoKR:
    def __init__(self, oc: Optional[str] = None, timeout: float = 20.0):
        self.base = "https://www.law.go.kr/DRF"
        self.oc = oc or os.getenv("LAW_OC")
        self.timeout = timeout
        self.sess = requests.Session()

    def _get(self, path: str, **params):
        if not self.oc:
            raise RuntimeError("No OC key configured for Law.go.kr API")
        q = {"OC": self.oc, **params}
        url = f"{self.base}/{path}"
        r = self.sess.get(url, params=q, timeout=self.timeout)
        r.raise_for_status()
        if "application/json" in (r.headers.get("Content-Type"," ").lower()):
            return r.json()
        return r.json()

    def search_law(self, query: str) -> Optional[Dict[str,Any]]:
        """query로 법령 1건 조회(내림차순 정렬 우선)"""
        try:
            data = self._get("lawSearch.do", target="law", type="JSON", query=query, display=1, sort="efdes")
            items = data.get("law") or data.get("LawSearch") or data
            if isinstance(items, dict) and "law" in items:
                items = items["law"]
            if isinstance(items, list) and items:
                item = items[0]
                return {
                    "official_name": item.get("법령명한글") or item.get("법령명"),
                    "ministry": item.get("소관부처명") or item.get("부처명"),
                    "recent_revision_date": item.get("공포일자") or item.get("시행일자"),
                    "lawId": item.get("법령ID")
                }
        except Exception:
            return None
        return None

    def get_article(self, lawId: str, art_no: Optional[str]=None) -> Optional[str]:
        """조문 예시를 1개 가져와 근거조문으로 제시(간이 파서)."""
        try:
            data = self._get("lawService.do", target="law", type="JSON", ID=lawId)
            txt = json.dumps(data, ensure_ascii=False)
            m = re.search(r"(제\s*\d+\s*조[^\"\\n]+)", txt)
            return m.group(1) if m else None
        except Exception:
            return None

def build_factbox(api: Optional[LawGoKR], law: LawEntry) -> Dict[str, Any]:
    """API가 있으면 보강, 없으면 카탈로그 메타로 생성"""
    fb = {
        "official_name": law.official_name,
        "ministry": None,
        "recent_revision_date": law.promulgation_date or law.effective_date,
        "evidence_article": None,
    }
    if api:
        meta = api.search_law(law.official_name)
        if meta:
            fb["ministry"] = meta.get("ministry") or fb["ministry"]
            fb["recent_revision_date"] = meta.get("recent_revision_date") or fb["recent_revision_date"]
            lawId = meta.get("lawId")
            if lawId:
                fb["evidence_article"] = api.get_article(lawId)
    if not fb["ministry"] and isinstance(law.raw_json, dict):
        for k in ["소관부처명","ministry","lawChnchrNm","lawKndCdnm"]:
            v = law.raw_json.get(k)
            if isinstance(v, str) and v:
                fb["ministry"] = v
                break
    return fb

In [5]:
# 4) Orchestrator — 파이프라인 본체
class LawLinker:
    def __init__(self, cfg: PipelineConfig):
        self.cfg = cfg
        # 1) 카탈로그 로드
        self.catalog: List[LawEntry] = load_law_catalog(cfg.law_json_path)
        # 2) BM25 인덱스
        self.bm25 = BM25LawCatalog(self.catalog)
        # 3) SBERT 재랭커
        self.rr = ReRanker(cfg.legal_sbert_model, device=cfg.device)
        # 4) NLI 추정기
        self.nli = NLIEstimator(cfg.nli_model, device=cfg.device)
        # 5) 법제처 API(optional)
        self.api = LawGoKR(cfg.law_api_oc) if cfg.use_law_api else None

    def _summarize_cluster(self, c: Cluster) -> str:
        """Step‑2: 키워드→문장화 + 대표문 일부를 전제로 사용"""
        if not c.summary_text:
            c.summary_text = sentences_from_keywords(c.keywords, k=20)
        rep = " ".join([normalize_text(x) for x in (c.rep_texts or [])[:2]])[:400]
        return (c.summary_text + " " + rep).strip()

    def _retrieve_candidates(self, query: str) -> List[Candidate]:
        """Step‑3: BM25@50 → SBERT 재랭킹@10"""
        bm25_hits = self.bm25.search(query, topk=self.cfg.bm25_topk)
        reranked = self.rr.rerank(query, bm25_hits, self.bm25.docs, topk=self.cfg.rerank_topk)
        bm25_map = {ix: sc for ix, sc in bm25_hits}
        out: List[Candidate] = []
        for ix, cos in reranked:
            law = self.catalog[ix]
            out.append(Candidate(law=law, bm25_score=bm25_map[ix], sbert_score=float(cos)))
        return out

    def _attach_factbox_and_conf(self, premise: str, cands: List[Candidate]) -> None:
        """Step‑4/5: NLI 엔테일 확률 + 팩트박스 + 신뢰도(conf) 계산"""
        if not cands:
            return
        svals = [c.sbert_score or 0.0 for c in cands]
        s_min, s_max = min(svals), max(svals)
        def norm(x):
            return 0.5 if abs(s_max - s_min) < 1e-9 else (x - s_min) / (s_max - s_min)
        for cand in cands:
            hyp = cand.law.official_name
            ent = self.nli.entail_prob(premise, hyp)
            cand.nli_entail_prob = ent
            cand.factbox = build_factbox(self.api, cand.law)
            rnorm = norm(cand.sbert_score or 0.0)
            cand.conf = 0.5 * rnorm + 0.5 * ent

    def _decision(self, cand: Candidate) -> str:
        if cand.conf is None:
            return "보류"
        if cand.conf >= self.cfg.auto_accept:
            return "자동 확정"
        lo, hi = self.cfg.human_review
        if lo <= cand.conf < hi:
            return "휴먼 검토"
        return "보류"

    def run_for_cluster(self, cluster: Cluster) -> Dict[str, Any]:
        if not cluster.ldvs_pass:
            return {"cluster_id": cluster.cluster_id, "skipped": True, "reason": "LDVS not passed"}
        premise = self._summarize_cluster(cluster)
        cands = self._retrieve_candidates(premise)
        self._attach_factbox_and_conf(premise, cands)
        cands = sorted(cands, key=lambda x: (x.conf or 0.0), reverse=True)
        out10 = [{
            "official_name": c.law.official_name,
            "bm25_score": c.bm25_score,
            "sbert_score": c.sbert_score,
            "nli_entail_prob": c.nli_entail_prob,
            "conf": c.conf,
            "decision": self._decision(c),
            "fact_box": c.factbox,
        } for c in cands]
        return {
            "cluster_id": cluster.cluster_id,
            "query": premise,
            "top1": out10[0] if out10 else None,
            "top3": out10[:3],
            "top10": out10
        }

    def run(self, clusters: List[Cluster]) -> List[Dict[str, Any]]:
        selected = filter_ldvs_clusters(clusters)
        return [self.run_for_cluster(c) for c in selected]

print('✅ Orchestrator ready')

✅ Orchestrator ready


In [10]:
# - 사용 필드: example_output.cluster_id, example_output.issue_summary.keywords
# - 선택 필드: example_output.issue_summary.representative_sentence → rep_texts에 활용(있으면 성능+)

import json, math, re
from pathlib import Path
from typing import Optional, Set, Iterable
import pandas as pd

JSON_PATH = "insta_priv_20251014_outputs.json"
# JSON_PATH = "../data/Clustering_Dataset/news/preprocess_news_part_1_with_metrics_outputs.json"
# JSON_PATH = "../data/Clustering_Dataset/SNS/파일이름.json"
XLSX_PATH = "preprocess_insta_part_1_table.xlsx"
# XLSX_PATH = "../data/Clustering_Dataset/news/preprocess_news_part_1_with_metrics_table.xlsx"
# XLSX_PATH = "../data/Clustering_Dataset/SNS/파일이름.xlsx"

# --- 유틸: 키워드 문자열을 리스트로 파싱 ---
_KW_PYLIST = re.compile(r"^\s*\[.+\]\s*$")
def _parse_keywords(cell) -> list[str]:
    """
    허용 포맷 예시:
      - 'a, b, c'
      - '핵심 키워드: a, b, c'
      - "['a','b','c']"
      - ['a','b','c'] (이미 list)
    """
    if cell is None or (isinstance(cell, float) and math.isnan(cell)):
        return []
    if isinstance(cell, list):
        return [str(x).strip() for x in cell if str(x).strip()]

    s = str(cell).strip()
    if not s:
        return []

    # '핵심 키워드:' 접두 제거
    s = re.sub(r"^\s*핵심\s*키워드\s*:\s*", "", s)

    # 파이썬 리스트 리터럴처럼 보이면 eval-safe 파싱
    if _KW_PYLIST.match(s):
        try:
            val = eval(s, {"__builtins__": None}, {})  # 단순 리터럴만 허용
            if isinstance(val, list):
                return [str(x).strip() for x in val if str(x).strip()]
        except Exception:
            pass

    # 콤마 분리 기본
    return [t.strip() for t in s.split(",") if t.strip()]


def _first_present(d: dict, keys: Iterable[str], default=None):
    for k in keys:
        if k in d and d[k] is not None and str(d[k]).strip() != "":
            return d[k]
    return default


def load_clusters_from_xlsx(
    path: str,
    sheet_name: Optional[str | int] = None,
    ldvs_pass_ids: Optional[Set[int]] = None,
    limit: Optional[int] = None,
) -> list[Cluster]:
    import pandas as pd
    x = pd.read_excel(path, sheet_name=sheet_name)  # sheet_name에 따라 DataFrame 또는 dict 반환

    # --- 시트 선택 처리 ---
    if isinstance(x, dict):
        # 사용자가 시트명을 문자열로 준 경우
        if isinstance(sheet_name, str) and sheet_name in x:
            df = x[sheet_name]
        # 사용자가 시트 인덱스를 정수로 준 경우
        elif isinstance(sheet_name, int):
            names = list(x.keys())
            idx = sheet_name if sheet_name >= 0 else len(names) + sheet_name
            if idx < 0 or idx >= len(names):
                raise IndexError(f"sheet_name index out of range: {sheet_name} (sheets={names})")
            df = x[names[idx]]
        else:
            # 명시가 없으면 첫 번째 '비어있지 않은' 시트를 사용
            df = next((sdf for sdf in x.values() if isinstance(sdf, pd.DataFrame) and not sdf.empty), None)
            if df is None:
                # 전부 비어있다면 첫 시트를 사용
                df = next(iter(x.values()))
    elif isinstance(x, pd.DataFrame):
        df = x
    else:
        raise TypeError(f"Unexpected type from read_excel: {type(x)}")

    # 컬럼명 소문자 매핑 사전
    cols_lc = {c.lower(): c for c in df.columns}

    # 후보 목록 정의
    cid_keys = ["cluster_id", "cid", "cluster", "id"]
    kw_keys  = ["keywords", "key_terms", "topk", "keyterms", "query", "키워드"]
    rep_keys = ["대표문장", "대표제목"]

    # 실제 존재하는 컬럼명 찾기
    cid_col = _first_present(cols_lc, cid_keys)
    kw_col  = _first_present(cols_lc, kw_keys)
    # rep는 여러 소스에서 모아볼 수 있도록 전부 후보로 순회
    rep_cols = [cols_lc[k] for k in rep_keys if k in cols_lc]

    clusters: list[Cluster] = []
    for _, row in df.iterrows():
        cid_raw = row.get(cid_col) if cid_col else None
        if cid_raw is None or (isinstance(cid_raw, float) and math.isnan(cid_raw)):
            continue
        try:
            cid = int(cid_raw)
        except Exception:
            # 숫자가 아니면 스킵
            continue

        # 키워드
        kw_cell = row.get(kw_col) if kw_col else None
        keywords = _parse_keywords(kw_cell)

        rep_texts: list[str] = []
        for rc in rep_cols:
            val = row.get(rc)
            if isinstance(val, list):
                rep_texts.extend([str(x).strip() for x in val if isinstance(x, str) and x.strip()])
            elif isinstance(val, str) and val.strip():
                # ';' 또는 '|' 로 여러 개가 들어온 경우도 처리
                parts = re.split(r"[|;]\s*", val.strip())
                rep_texts.extend([p for p in parts if p])

        rep_texts = []  # dedupe는 필요시 수행

        ldvs_ok = True if ldvs_pass_ids is None else (cid in ldvs_pass_ids)
        clusters.append(Cluster(
            cluster_id=cid,
            ldvs_pass=ldvs_ok,
            keywords=keywords,
            rep_texts=rep_texts
        ))
        if limit is not None and len(clusters) >= limit:
            break

    print(f"📗 Loaded clusters from XLSX: {len(clusters)}")
    for c in clusters[:2]:
        print(f" - cid={c.cluster_id}, ldvs={c.ldvs_pass}, kws(sample)={c.keywords[:8]}")
    return clusters

def load_clusters(
    path: str,
    ldvs_pass_ids: Optional[Set[int]] = None,
    limit: Optional[int] = None,
    sheet_name: Optional[str] = None,
) -> list[Cluster]:
    """
    통합 로더: 확장자에 따라 JSON / XLSX 자동 처리
    - JSON: 기존 함수 로직 그대로 재사용
    - XLSX: 위의 엑셀 로더 사용
    """
    ext = Path(path).suffix.lower()
    if ext in [".xlsx", ".xls"]:
        return load_clusters_from_xlsx(path, sheet_name=sheet_name, ldvs_pass_ids=ldvs_pass_ids, limit=limit)

    # ----- 기존 JSON 로직 (필요시 그대로 둠) -----
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    clusters: list[Cluster] = []

    if isinstance(data, list):
        for row in data:
            eo = (row or {}).get("example_output", {}) or {}
            cid = eo.get("cluster_id")
            if cid is None:
                continue
            isu = eo.get("issue_summary", {}) or {}
            keywords = isu.get("keywords", []) or []
            rep = []
            rep_texts = [rep] if isinstance(rep, str) and rep.strip() else []

            ldvs_ok = True if ldvs_pass_ids is None else (cid in ldvs_pass_ids)
            clusters.append(Cluster(
                cluster_id=int(cid),
                ldvs_pass=ldvs_ok,
                keywords=[str(k).strip() for k in keywords if str(k).strip()],
                rep_texts=rep_texts
            ))
            if limit is not None and len(clusters) >= limit:
                break

    elif isinstance(data, dict) and "top_clusters" in data:
        for item in (data.get("top_clusters") or []):
            cid = item.get("cluster_id")
            if cid is None:
                continue
            kw_raw = item.get("keywords", "")
            if isinstance(kw_raw, str):
                keywords = [k.strip() for k in kw_raw.split(",") if k.strip()]
            elif isinstance(kw_raw, list):
                keywords = [str(k).strip() for k in kw_raw if str(k).strip()]
            else:
                keywords = []

            rep_titles = []
            rep_texts = [t for t in rep_titles if isinstance(t, str) and t.strip()]

            ldvs_ok = True if ldvs_pass_ids is None else (cid in ldvs_pass_ids)
            clusters.append(Cluster(
                cluster_id=int(cid),
                ldvs_pass=ldvs_ok,
                keywords=keywords,
                rep_texts=rep_texts
            ))
            if limit is not None and len(clusters) >= limit:
                break
    else:
        raise ValueError("Unsupported schema: expected list[...] or dict with 'top_clusters'.")

    print(f"📦 Loaded clusters from JSON: {len(clusters)}")
    for c in clusters[:2]:
        print(f" - cid={c.cluster_id}, ldvs={c.ldvs_pass}, kws(sample)={c.keywords[:8]}")
    return clusters

# 5.1 (옵션) LDVS 통과 집합이 따로 있다면 여기에 지정하세요.
# 예: ldvs_pass_ids = {0,1,3,4,5,10}
ldvs_pass_ids = None

# 5.2 파이프라인 초기화(모델/경로는 위 셀(cfg)에서 조정)
linker = LawLinker(cfg)

# 5.3 엑셀에서 클러스터 불러오기
clusters = load_clusters(XLSX_PATH, ldvs_pass_ids=ldvs_pass_ids, limit=50, sheet_name=None)

# 5.4 실행
results = linker.run(clusters)

from pprint import pprint
pprint(results[:2])  # 출력이 길 수 있어 앞쪽 일부만 미리보기
print("\n🎯 Done — 각 cluster의 top1/top3/top10, decision(자동 확정/휴먼 검토/보류), fact_box를 확인하세요.")

🔍 JSON top-level shape:
list(len=1)
  [0]: dict
    dict(keys=['stmt', 'header', 'rows'])
      - stmt: str
      - header: list
      - rows: list
        str
📎 Found table (200 rows) via nested search
📚 Loaded laws: 200


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


📗 Loaded clusters from XLSX: 34
 - cid=35, ldvs=True, kws(sample)=['상에', '악의적', '무분별하게', '경고드립니다', '확산되고', '느끼고', '강경', '행위입니다']
 - cid=22, ldvs=True, kws(sample)=['공수처', '정동훈', '다녀왔다', '노상원', '수사하', '수사해야', '유출됐다며', '특검팀']
[{'cluster_id': 35,
  'query': '핵심 키워드: 상에, 악의적, 무분별하게, 경고드립니다, 확산되고, 느끼고, 강경, 행위입니다, 대중, 제307조, '
           '가벼, 여겨져서, 아티스트, 갤럭시코퍼레이션, 게시글.',
  'top1': {'bm25_score': 0.0,
           'conf': 0.641081228852272,
           'decision': '휴먼 검토',
           'fact_box': {'evidence_article': None,
                        'ministry': '환경보건법 시행령',
                        'official_name': '환경보건법 시행령',
                        'recent_revision_date': '20250826'},
           'nli_entail_prob': 0.28216245770454407,
           'official_name': '환경보건법 시행령',
           'sbert_score': 0.4125846028327942},
  'top10': [{'bm25_score': 0.0,
             'conf': 0.641081228852272,
             'decision': '휴먼 검토',
             'fact_box': {'evidence_article': None,
                     

In [11]:
# 6) Export — 최종 맵핑 결과 CSV 저장 (Top-1 / Top-3)

import pandas as pd
from typing import List, Dict, Any, Optional

# (선택) cluster_id → 원본 cluster 메타(키워드/대표문) 조회에 사용
_cluster_map = {c.cluster_id: c for c in clusters} if 'clusters' in globals() else {}

def _get_cluster_meta(cid: int) -> Dict[str, Any]:
    c = _cluster_map.get(cid)
    if not c:
        return {}
    return {
        "cluster_keywords": ", ".join(c.keywords[:20]) if getattr(c, "keywords", None) else None,
        "cluster_rep_texts": " | ".join(c.rep_texts[:2]) if getattr(c, "rep_texts", None) else None,
    }

def to_top1_df(results: List[Dict[str, Any]]) -> pd.DataFrame:
    """각 클러스터의 채택 후보 Top-1만 요약"""
    rows = []
    for r in results:
        if r.get("skipped"):  # LDVS 탈락 등
            continue
        top1 = r.get("top1")
        if not top1:
            continue
        fb = top1.get("fact_box") or {}
        meta = _get_cluster_meta(r["cluster_id"])
        rows.append({
            "cluster_id": r["cluster_id"],
            "query": r.get("query"),
            "official_name": top1.get("official_name"),
            "decision": top1.get("decision"),
            "conf": round((top1.get("conf") or 0.0), 4),
            "nli_entail_prob": round((top1.get("nli_entail_prob") or 0.0), 4),
            "sbert_score": round((top1.get("sbert_score") or 0.0), 4),
            "bm25_score": round((top1.get("bm25_score") or 0.0), 4),
            "ministry": fb.get("ministry"),
            "recent_revision_date": fb.get("recent_revision_date"),
            "evidence_article": fb.get("evidence_article"),
            **meta
        })
    return pd.DataFrame(rows).sort_values(["cluster_id"]).reset_index(drop=True)

def to_topk_df(results: List[Dict[str, Any]], k: int = 3) -> pd.DataFrame:
    """각 클러스터에서 상위 k개 후보를 롱포맷으로 저장(rank=1..k)"""
    rows = []
    for r in results:
        if r.get("skipped"):
            continue
        meta = _get_cluster_meta(r["cluster_id"])
        for rank, item in enumerate((r.get("top10") or [])[:k], start=1):
            fb = item.get("fact_box") or {}
            rows.append({
                "cluster_id": r["cluster_id"],
                "rank": rank,
                "query": r.get("query"),
                "official_name": item.get("official_name"),
                "decision": item.get("decision"),
                "conf": round((item.get("conf") or 0.0), 4),
                "nli_entail_prob": round((item.get("nli_entail_prob") or 0.0), 4),
                "sbert_score": round((item.get("sbert_score") or 0.0), 4),
                "bm25_score": round((item.get("bm25_score") or 0.0), 4),
                "ministry": fb.get("ministry"),
                "recent_revision_date": fb.get("recent_revision_date"),
                "evidence_article": fb.get("evidence_article"),
                **meta
            })
    return pd.DataFrame(rows).sort_values(["cluster_id", "rank"]).reset_index(drop=True)

# 변환
top1_df = to_top1_df(results)
top3_df = to_topk_df(results, k=3)

# 저장 경로
top1_path = "law_mapping_top1.csv"
top3_path = "law_mapping_top3.csv"

# CSV 저장 (엑셀 한글 깨짐 방지: utf-8-sig)
top1_df.to_csv(top1_path, index=False, encoding="utf-8-sig")
top3_df.to_csv(top3_path, index=False, encoding="utf-8-sig")

print(f"✅ Saved:\n - {top1_path}\n - {top3_path}")

# 미리보기 (행 수가 많을 수 있으니 head만)
display(top1_df.head(10))
display(top3_df.head(10))


✅ Saved:
 - law_mapping_top1.csv
 - law_mapping_top3.csv


,cluster_id,query,official_name,decision,conf,nli_entail_prob,sbert_score,bm25_score,ministry,recent_revision_date,evidence_article,cluster_keywords,cluster_rep_texts
0,0,"핵심 키워드: 피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참...",방송법,자동 확정,0.8648,0.7297,0.2644,0.0,방송법,20250826,None,"피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참았던, 이젠, ...",None
1,2,"핵심 키워드: 아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하...",지방세특례제한법 시행령,휴먼 검토,0.7490,0.4980,0.3320,0.0,지방세특례제한법 시행령,20250826,None,"아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하고, 어라운드스...",None
2,3,"핵심 키워드: 게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아...",방송법,자동 확정,0.8092,0.6185,0.3956,0.0,방송법,20250826,None,"게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아이캔아이엠위캔위...",None
3,4,"핵심 키워드: 유출, 보안, 담당업무, 악성코드, skt, 가입, 복제, 공격, w...",공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,휴먼 검토,0.7430,0.4861,0.4081,0.0,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,20250819,None,"유출, 보안, 담당업무, 악성코드, skt, 가입, 복제, 공격, world, 차단...",None
4,6,"핵심 키워드: 벌점, 모니터, 설치할, 영화, 근미래, 감시하고, 개인영상정보, 사...",선거관리위원회 공무원 규칙,자동 확정,0.7513,0.5026,0.2597,0.0,선거관리위원회 공무원 규칙,20250821,None,"벌점, 모니터, 설치할, 영화, 근미래, 감시하고, 개인영상정보, 사카모토, 보여줄...",None
5,8,"핵심 키워드: 흥덕, 상담사, 어려울것, 원서, 도시라, 모의면접, 찻값, 금전, ...",우편법 시행규칙,휴먼 검토,0.6789,0.3579,0.3067,0.0,우편법 시행규칙,20250818,None,"흥덕, 상담사, 어려울것, 원서, 도시라, 모의면접, 찻값, 금전, 청두에, 인민공...",None
6,9,"핵심 키워드: 교육청에, 학폭, 수험생, 예방교육, 법정의무교육강사, 장애인인식개선...",헌법재판소 공무원 규칙,휴먼 검토,0.6658,0.3316,0.3460,0.0,헌법재판소 공무원 규칙,20250821,None,"교육청에, 학폭, 수험생, 예방교육, 법정의무교육강사, 장애인인식개선교육, 에듀이너...",None
7,11,"핵심 키워드: 글귀, 신청방법, 원데이클래스, 7층, 청춘정거장, 실습, 강의장소,...",지방세특례제한법 시행령,휴먼 검토,0.7325,0.4651,0.4085,0.0,지방세특례제한법 시행령,20250826,None,"글귀, 신청방법, 원데이클래스, 7층, 청춘정거장, 실습, 강의장소, 198, 15...",None
8,12,"핵심 키워드: 수급자, 청년성장프로젝트, 견학, 성공취업, 34세, 20명, 이동,...",지방세특례제한법 시행령,자동 확정,0.7930,0.5859,0.4185,0.0,지방세특례제한법 시행령,20250826,None,"수급자, 청년성장프로젝트, 견학, 성공취업, 34세, 20명, 이동, 관심있, 집결...",None
9,13,"핵심 키워드: 방면, heraldbiz, 일간베스트, 헤럴드경제, 518민주화운동,...",지방세특례제한법 시행령,휴먼 검토,0.6745,0.3490,0.3649,0.0,지방세특례제한법 시행령,20250826,None,"방면, heraldbiz, 일간베스트, 헤럴드경제, 518민주화운동, 더메디컬, 랭...",None


,cluster_id,rank,query,official_name,decision,conf,nli_entail_prob,sbert_score,bm25_score,ministry,recent_revision_date,evidence_article,cluster_keywords,cluster_rep_texts
0,0,1,"핵심 키워드: 피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참...",방송법,자동 확정,0.8648,0.7297,0.2644,0.0,방송법,20250826,None,"피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참았던, 이젠, ...",None
1,0,2,"핵심 키워드: 피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참...",공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,휴먼 검토,0.6042,0.4946,0.2413,0.0,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,20250819,None,"피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참았던, 이젠, ...",None
2,0,3,"핵심 키워드: 피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참...",우편법 시행규칙,보류,0.5264,0.2950,0.2449,0.0,우편법 시행규칙,20250818,None,"피임, 상담해준대, 성건강상담, 카톡채널, 지원정책, 러브플랜에, 참았던, 이젠, ...",None
3,2,1,"핵심 키워드: 아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하...",지방세특례제한법 시행령,휴먼 검토,0.7490,0.4980,0.3320,0.0,지방세특례제한법 시행령,20250826,None,"아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하고, 어라운드스...",None
4,2,2,"핵심 키워드: 아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하...",공무원연금법 시행령,보류,0.5162,0.4383,0.3147,0.0,공무원연금법 시행령,20250826,None,"아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하고, 어라운드스...",None
5,2,3,"핵심 키워드: 아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하...",주택도시기금법 시행령,보류,0.3805,0.4577,0.3023,0.0,주택도시기금법 시행령,20250826,None,"아리엘음악치료센터아리의음악학원아리작곡, 성장하, 창의감성교육, 준수하고, 어라운드스...",None
6,3,1,"핵심 키워드: 게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아...",방송법,자동 확정,0.8092,0.6185,0.3956,0.0,방송법,20250826,None,"게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아이캔아이엠위캔위...",None
7,3,2,"핵심 키워드: 게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아...",공무원연금법 시행령,휴먼 검토,0.6398,0.4320,0.3838,0.0,공무원연금법 시행령,20250826,None,"게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아이캔아이엠위캔위...",None
8,3,3,"핵심 키워드: 게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아...",방송통신위원회의 설치 및 운영에 관한 법률,보류,0.5037,0.4878,0.3585,0.0,방송통신위원회의 설치 및 운영에 관한 법률,20250826,None,"게시됩니다, 가족합창단발달장애자폐스펙트럼지적장애, 자녀들, 아임소리아이캔아이엠위캔위...",None
9,4,1,"핵심 키워드: 유출, 보안, 담당업무, 악성코드, skt, 가입, 복제, 공격, w...",공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,휴먼 검토,0.7430,0.4861,0.4081,0.0,공문서에 대한 아포스티유 및 본부영사확인서 발급에 관한 규정 시행규칙,20250819,None,"유출, 보안, 담당업무, 악성코드, skt, 가입, 복제, 공격, world, 차단...",None
